# Iowa Election Results Downloader (2016, 2018, 2020)

This notebook downloads precinct-level election results for all 99 Iowa counties across three election years:
- **2016 General Election** (.xlsx files)
- **2018 General Election** (.xls files)
- **2020 General Election** (.xlsx files)

**Total files to download:** 297 files (99 counties × 3 years)

**Source:** Iowa Secretary of State - https://sos.iowa.gov/

## 2. Import Libraries

In [7]:
import os
import time
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## 3. Iowa Counties List

All 99 Iowa counties in alphabetical order.

In [8]:
# Iowa's 99 counties (in alphabetical order)
IOWA_COUNTIES = [
    "Adair", "Adams", "Allamakee", "Appanoose", "Audubon",
    "Benton", "Black Hawk", "Boone", "Bremer", "Buchanan",
    "Buena Vista", "Butler", "Calhoun", "Carroll", "Cass",
    "Cedar", "Cerro Gordo", "Cherokee", "Chickasaw", "Clarke",
    "Clay", "Clayton", "Clinton", "Crawford", "Dallas",
    "Davis", "Decatur", "Delaware", "Des Moines", "Dickinson",
    "Dubuque", "Emmet", "Fayette", "Floyd", "Franklin",
    "Fremont", "Greene", "Grundy", "Guthrie", "Hamilton",
    "Hancock", "Hardin", "Harrison", "Henry", "Howard",
    "Humboldt", "Ida", "Iowa", "Jackson", "Jasper",
    "Jefferson", "Johnson", "Jones", "Keokuk", "Kossuth",
    "Lee", "Linn", "Louisa", "Lucas", "Lyon",
    "Madison", "Mahaska", "Marion", "Marshall", "Mills",
    "Mitchell", "Monona", "Monroe", "Montgomery", "Muscatine",
    "O'Brien", "Osceola", "Page", "Palo Alto", "Plymouth",
    "Pocahontas", "Polk", "Pottawattamie", "Poweshiek", "Ringgold",
    "Sac", "Scott", "Shelby", "Sioux", "Story",
    "Tama", "Taylor", "Union", "Van Buren", "Wapello",
    "Warren", "Washington", "Wayne", "Webster", "Winnebago",
    "Winneshiek", "Woodbury", "Worth", "Wright"
]

print(f"Iowa has {len(IOWA_COUNTIES)} counties")
print(f"First few: {IOWA_COUNTIES[:5]}")
print(f"Last few: {IOWA_COUNTIES[-5:]}")

Iowa has 99 counties
First few: ['Adair', 'Adams', 'Allamakee', 'Appanoose', 'Audubon']
Last few: ['Winnebago', 'Winneshiek', 'Woodbury', 'Worth', 'Wright']


## 4. Configuration

Choose your browser and set up directories.

In [9]:
# Choose your browser: "chrome", "edge", or "firefox"
BROWSER = "chrome"  # Change this if you want to use a different browser

# Setup directories
OUTPUT_DIR = Path("./iowa_election_results")
TEMP_DIR = OUTPUT_DIR / "_temp"
OUTPUT_DIR.mkdir(exist_ok=True)
TEMP_DIR.mkdir(exist_ok=True)

print(f"Browser: {BROWSER}")
print(f"Output directory: {OUTPUT_DIR.absolute()}")
print(f"Temp directory: {TEMP_DIR.absolute()}")

Browser: chrome
Output directory: c:\Users\18607\Downloads\iowa_election_results
Temp directory: c:\Users\18607\Downloads\iowa_election_results\_temp


## 5. Helper Functions

Functions to handle downloads and file management.

In [10]:
def wait_for_download(timeout=60):
    """Wait until download completes."""
    start = time.time()
    while time.time() - start < timeout:
        files = list(TEMP_DIR.iterdir())
        partial = [f for f in files if f.suffix in ('.crdownload', '.tmp', '.part')]
        done = [f for f in files if f.suffix in ('.xls', '.xlsx', '.pdf')]
        if done and not partial:
            return done[0]
        time.sleep(0.5)
    return None

def clear_temp():
    """Empty the temp folder."""
    for f in TEMP_DIR.iterdir():
        try:
            f.unlink()
        except:
            pass

print("✓ Helper functions defined")

✓ Helper functions defined


## 6. Start Browser

Initialize the Selenium browser with download preferences.

**Note:** A browser window will open. Don't close it until all downloads are complete!

In [11]:
# Start the browser
browser = BROWSER.lower()

if browser == "chrome":
    from selenium.webdriver.chrome.options import Options
    options = Options()
    prefs = {
        "download.default_directory": str(TEMP_DIR.absolute()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
elif browser == "edge":
    from selenium.webdriver.edge.options import Options
    from webdriver_manager.microsoft import EdgeChromiumDriverManager
    options = Options()
    prefs = {
        "download.default_directory": str(TEMP_DIR.absolute()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    
elif browser == "firefox":
    from selenium.webdriver.firefox.options import Options
    from webdriver_manager.firefox import GeckoDriverManager
    options = Options()
    options.set_preference("browser.download.dir", str(TEMP_DIR.absolute()))
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.useDownloadDir", True)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", 
                          "application/vnd.ms-excel,application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

else:
    raise ValueError(f"Unknown browser: {browser}. Choose 'chrome', 'edge', or 'firefox'")

print(f"✓ Started {browser} browser")
print(f"Files will be saved to: {OUTPUT_DIR.absolute()}\n")

✓ Started chrome browser
Files will be saved to: c:\Users\18607\Downloads\iowa_election_results



## 7. Download 2016 Files (.xlsx)

Downloading all 99 counties for the 2016 General Election.

**URL Pattern:** `https://sos.iowa.gov/elections/pdf/precinctresults/2016general/{county}.xlsx`

**Note:** County names in URLs are lowercase and keep spaces/apostrophes (e.g., "black hawk", "o'brien")

In [12]:
print("=" * 50)
print("Downloading 2016 files...")
print("=" * 50)

successful_2016 = 0
failed_2016 = []

for i, county in enumerate(IOWA_COUNTIES, 1):
    # County name: lowercase, KEEP spaces and apostrophes
    # Browser will handle URL encoding automatically
    county_url = county.lower()
    
    # 2016 uses .xlsx format
    url = f"https://sos.iowa.gov/elections/pdf/precinctresults/2016general/{county_url}.xlsx"
    
    print(f"[{i}/99] {county}...", end=" ")
    
    clear_temp()
    driver.get(url)
    downloaded = wait_for_download()
    
    if downloaded:
        new_name = f"{county}_2016.xlsx"
        downloaded.rename(OUTPUT_DIR / new_name)
        print("✓")
        successful_2016 += 1
    else:
        print("✗ FAILED")
        failed_2016.append(county)
    
    time.sleep(0.5)

print(f"\n2016 Complete: {successful_2016} successful, {len(failed_2016)} failed")
if failed_2016:
    print(f"Failed counties: {', '.join(failed_2016)}")

[1/99] Adair... ✓
[2/99] Adams... ✓
[3/99] Allamakee... ✓
[4/99] Appanoose... ✓
[5/99] Audubon... ✓
[6/99] Benton... ✓
[7/99] Black Hawk... ✓
[8/99] Boone... ✓
[9/99] Bremer... ✓
[10/99] Buchanan... ✓
[11/99] Buena Vista... ✓
[12/99] Butler... ✓
[13/99] Calhoun... ✓
[14/99] Carroll... ✓
[15/99] Cass... ✓
[16/99] Cedar... ✓
[17/99] Cerro Gordo... ✓
[18/99] Cherokee... ✓
[19/99] Chickasaw... ✓
[20/99] Clarke... ✓
[21/99] Clay... ✓
[22/99] Clayton... ✓
[23/99] Clinton... ✓
[24/99] Crawford... ✓
[25/99] Dallas... ✓
[26/99] Davis... ✓
[27/99] Decatur... ✓
[28/99] Delaware... ✓
[29/99] Des Moines... ✓
[30/99] Dickinson... ✓
[31/99] Dubuque... ✓
[32/99] Emmet... ✓
[33/99] Fayette... ✓
[34/99] Floyd... ✓
[35/99] Franklin... ✓
[36/99] Fremont... ✓
[37/99] Greene... ✓
[38/99] Grundy... ✓
[39/99] Guthrie... ✓
[40/99] Hamilton... ✓
[41/99] Hancock... ✓
[42/99] Hardin... ✓
[43/99] Harrison... ✓
[44/99] Henry... ✓
[45/99] Howard... ✓
[46/99] Humboldt... ✓
[47/99] Ida... ✓
[48/99] Iowa... ✓
[49/99] J

## 8. Download 2018 Files (.xls)

Downloading all 99 counties for the 2018 General Election.

**URL Pattern:** `https://sos.iowa.gov/elections/pdf/precinctresults/2018general/{county}.xls`

**Note:** 2018 uses the older .xls format (not .xlsx)

In [13]:
print("\n" + "=" * 50)
print("Downloading 2018 files...")
print("=" * 50)

successful_2018 = 0
failed_2018 = []

for i, county in enumerate(IOWA_COUNTIES, 1):
    # County name: lowercase, KEEP spaces and apostrophes
    county_url = county.lower()
    
    # 2018 uses .xls format (older Excel format)
    url = f"https://sos.iowa.gov/elections/pdf/precinctresults/2018general/{county_url}.xls"
    
    print(f"[{i}/99] {county}...", end=" ")
    
    clear_temp()
    driver.get(url)
    downloaded = wait_for_download()
    
    if downloaded:
        new_name = f"{county}_2018.xls"
        downloaded.rename(OUTPUT_DIR / new_name)
        print("✓")
        successful_2018 += 1
    else:
        print("✗ FAILED")
        failed_2018.append(county)
    
    time.sleep(0.5)

print(f"\n2018 Complete: {successful_2018} successful, {len(failed_2018)} failed")
if failed_2018:
    print(f"Failed counties: {', '.join(failed_2018)}")


[1/99] Adair... ✓
[2/99] Adams... ✓
[3/99] Allamakee... ✓
[4/99] Appanoose... ✓
[5/99] Audubon... ✓
[6/99] Benton... ✓
[7/99] Black Hawk... ✓
[8/99] Boone... ✓
[9/99] Bremer... ✓
[10/99] Buchanan... ✓
[11/99] Buena Vista... ✓
[12/99] Butler... ✓
[13/99] Calhoun... ✓
[14/99] Carroll... ✓
[15/99] Cass... ✓
[16/99] Cedar... ✓
[17/99] Cerro Gordo... ✓
[18/99] Cherokee... ✓
[19/99] Chickasaw... ✓
[20/99] Clarke... ✓
[21/99] Clay... ✓
[22/99] Clayton... ✓
[23/99] Clinton... ✓
[24/99] Crawford... ✓
[25/99] Dallas... ✓
[26/99] Davis... ✓
[27/99] Decatur... ✓
[28/99] Delaware... ✓
[29/99] Des Moines... ✓
[30/99] Dickinson... ✓
[31/99] Dubuque... ✓
[32/99] Emmet... ✓
[33/99] Fayette... ✓
[34/99] Floyd... ✓
[35/99] Franklin... ✓
[36/99] Fremont... ✓
[37/99] Greene... ✓
[38/99] Grundy... ✓
[39/99] Guthrie... ✓
[40/99] Hamilton... ✓
[41/99] Hancock... ✓
[42/99] Hardin... ✓
[43/99] Harrison... ✓
[44/99] Henry... ✓
[45/99] Howard... ✓
[46/99] Humboldt... ✓
[47/99] Ida... ✓
[48/99] Iowa... ✓
[49/99] 

## 9. Download 2020 Files (.xlsx)

Downloading all 99 counties for the 2020 General Election.

**URL Pattern:** `https://sos.iowa.gov/elections/pdf/precinctresults/2020general/{county}.xlsx`

**Special case:** Scott County 2020 is available as a PDF instead of Excel

In [14]:
print("\n" + "=" * 50)
print("Downloading 2020 files...")
print("=" * 50)

successful_2020 = 0
failed_2020 = []

for i, county in enumerate(IOWA_COUNTIES, 1):
    county_url = county.lower()
    
    # Scott County 2020 is a PDF, all others are XLSX
    if county == "Scott":
        extension = "pdf"
    else:
        extension = "xlsx"
    
    url = f"https://sos.iowa.gov/elections/pdf/precinctresults/2020general/{county_url}.{extension}"
    
    print(f"[{i}/99] {county}...", end=" ")
    
    clear_temp()
    driver.get(url)
    downloaded = wait_for_download()
    
    if downloaded:
        new_name = f"{county}_2020.{extension}"
        downloaded.rename(OUTPUT_DIR / new_name)
        if extension == "pdf":
            print("✓ (PDF)")
        else:
            print("✓")
        successful_2020 += 1
    else:
        print("✗ FAILED")
        failed_2020.append(county)
    
    time.sleep(0.5)

print(f"\n2020 Complete: {successful_2020} successful, {len(failed_2020)} failed")
if failed_2020:
    print(f"Failed counties: {', '.join(failed_2020)}")


[1/99] Adair... ✓
[2/99] Adams... ✓
[3/99] Allamakee... ✓
[4/99] Appanoose... ✓
[5/99] Audubon... ✓
[6/99] Benton... ✓
[7/99] Black Hawk... ✓
[8/99] Boone... ✓
[9/99] Bremer... ✓
[10/99] Buchanan... ✓
[11/99] Buena Vista... ✓
[12/99] Butler... ✓
[13/99] Calhoun... ✓
[14/99] Carroll... ✓
[15/99] Cass... ✓
[16/99] Cedar... ✓
[17/99] Cerro Gordo... ✓
[18/99] Cherokee... ✓
[19/99] Chickasaw... ✓
[20/99] Clarke... ✓
[21/99] Clay... ✓
[22/99] Clayton... ✓
[23/99] Clinton... ✓
[24/99] Crawford... ✓
[25/99] Dallas... ✓
[26/99] Davis... ✓
[27/99] Decatur... ✓
[28/99] Delaware... ✓
[29/99] Des Moines... ✓
[30/99] Dickinson... ✓
[31/99] Dubuque... ✓
[32/99] Emmet... ✓
[33/99] Fayette... ✓
[34/99] Floyd... ✓
[35/99] Franklin... ✓
[36/99] Fremont... ✓
[37/99] Greene... ✓
[38/99] Grundy... ✓
[39/99] Guthrie... ✓
[40/99] Hamilton... ✓
[41/99] Hancock... ✓
[42/99] Hardin... ✓
[43/99] Harrison... ✓
[44/99] Henry... ✓
[45/99] Howard... ✓
[46/99] Humboldt... ✓
[47/99] Ida... ✓
[48/99] Iowa... ✓
[49/99] 

## 10. Close Browser and Summary

In [15]:
# Close the browser
driver.quit()
print("\n✓ Browser closed")

# Summary
print("\n" + "=" * 50)
print("DOWNLOAD SUMMARY")
print("=" * 50)
print(f"2016: {successful_2016}/99 successful")
print(f"2018: {successful_2018}/99 successful")
print(f"2020: {successful_2020}/99 successful")
print(f"\nTotal: {successful_2016 + successful_2018 + successful_2020}/297 files downloaded")

total_failed = len(failed_2016) + len(failed_2018) + len(failed_2020)
if total_failed == 0:
    print("\n✓✓✓ SUCCESS! All 297 files downloaded! ✓✓✓")
else:
    print(f"\n⚠ {total_failed} files failed")
    if failed_2016:
        print(f"   2016 failed: {', '.join(failed_2016)}")
    if failed_2018:
        print(f"   2018 failed: {', '.join(failed_2018)}")
    if failed_2020:
        print(f"   2020 failed: {', '.join(failed_2020)}")

print(f"\nFiles saved to: {OUTPUT_DIR.absolute()}")
print("=" * 50)


✓ Browser closed

DOWNLOAD SUMMARY
2016: 99/99 successful
2018: 99/99 successful
2020: 98/99 successful

Total: 296/297 files downloaded

⚠ 1 files failed
   2020 failed: Scott

Files saved to: c:\Users\18607\Downloads\iowa_election_results


## 11. Verify Downloads

Check what files were downloaded.

In [16]:
# List all downloaded files
files = sorted([f for f in OUTPUT_DIR.iterdir() if f.suffix in ('.xls', '.xlsx', '.pdf')])

print(f"Total files in output directory: {len(files)}\n")

# Count by year
files_2016 = [f for f in files if '2016' in f.name]
files_2018 = [f for f in files if '2018' in f.name]
files_2020 = [f for f in files if '2020' in f.name]

print(f"2016 files: {len(files_2016)}")
print(f"2018 files: {len(files_2018)}")
print(f"2020 files: {len(files_2020)}")

# Calculate total size
total_size_mb = sum(f.stat().st_size for f in files) / (1024 * 1024)
print(f"\nTotal size: {total_size_mb:.2f} MB")

# Show first few files from each year
print("\nSample files:")
print("  2016:", [f.name for f in files_2016[:3]])
print("  2018:", [f.name for f in files_2018[:3]])
print("  2020:", [f.name for f in files_2020[:3]])

Total files in output directory: 296

2016 files: 99
2018 files: 99
2020 files: 98

Total size: 87.65 MB

Sample files:
  2016: ['Adair_2016.xlsx', 'Adams_2016.xlsx', 'Allamakee_2016.xlsx']
  2018: ['Adair_2018.xls', 'Adams_2018.xls', 'Allamakee_2018.xls']
  2020: ['Adair_2020.xlsx', 'Adams_2020.xlsx', 'Allamakee_2020.xlsx']


## 12. Clean Up Temp Directory (Optional)

In [17]:
# Remove the temporary directory
import shutil

try:
    shutil.rmtree(TEMP_DIR)
    print(f"✓ Temporary directory removed: {TEMP_DIR}")
except Exception as e:
    print(f"Could not remove temp directory: {e}")

✓ Temporary directory removed: iowa_election_results\_temp
